<a href="https://colab.research.google.com/github/srnarasim/DataProcessingComparison/blob/main/scenario2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scenario 2: The Production ETL Pipeline

**Constraints**: Reliability, monitoring, error handling, integration with existing infrastructure

This notebook demonstrates how different data processing tools handle production ETL scenarios with:
- Large daily data volumes (50GB+ simulated with smaller datasets)
- Fault tolerance requirements
- Complex transformations
- Error handling and monitoring

We'll compare **Spark**, **Polars**, **Pandas**, and **DuckDB** for production ETL workloads.

In [ ]:
# Install required packages for production ETL scenario
!pip install polars duckdb pyarrow pyspark findspark
!apt-get update -qq
!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/.local/lib/python3.10/site-packages/pyspark"

In [ ]:
# Import libraries and setup
import pandas as pd
import polars as pl
import duckdb
import numpy as np
import time
import logging
import glob
from datetime import datetime, timedelta
from pathlib import Path

# Setup Spark
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("All libraries imported successfully!")

In [ ]:
# Generate realistic production-scale sample data
def generate_production_data(n_rows=1_000_000, n_files=5):
    """Generate multiple files simulating daily transaction batches"""
    np.random.seed(42)
    
    # Create data directory
    Path("data/transactions/2025/01").mkdir(parents=True, exist_ok=True)
    
    customers = [f"CUST_{i:06d}" for i in range(1, 50001)]  # 50K customers
    products = [f"PROD_{i:04d}" for i in range(1, 5001)]   # 5K products
    categories = ['Electronics', 'Clothing', 'Books', 'Home', 'Sports', 'Beauty']
    
    files_created = []
    
    for file_idx in range(n_files):
        # Simulate daily batch with some variation
        daily_rows = n_rows + np.random.randint(-50000, 50000)
        base_date = datetime(2025, 1, 1) + timedelta(days=file_idx)
        
        data = {
            'order_id': [f"ORD_{file_idx}_{i:08d}" for i in range(daily_rows)],
            'customer_id': np.random.choice(customers, daily_rows),
            'product_id': np.random.choice(products, daily_rows),
            'product_category': np.random.choice(categories, daily_rows),
            'order_total': np.random.lognormal(3, 1, daily_rows).round(2),
            'order_date': [
                base_date + timedelta(hours=np.random.randint(0, 24),
                                    minutes=np.random.randint(0, 60))
                for _ in range(daily_rows)
            ],
            'processing_batch': f"2025-01-{file_idx+1:02d}"
        }
        
        # Add some corrupted data to test error handling (5% of records)
        corruption_indices = np.random.choice(daily_rows, size=int(daily_rows * 0.05), replace=False)
        for idx in corruption_indices:
            if np.random.random() < 0.5:
                data['order_total'][idx] = None  # Missing values
            else:
                data['customer_id'][idx] = "INVALID_CUSTOMER"  # Invalid customer
        
        df = pd.DataFrame(data)
        filename = f"data/transactions/2025/01/transactions_2025-01-{file_idx+1:02d}.csv"
        df.to_csv(filename, index=False)
        files_created.append(filename)
        
        print(f"Created {filename} with {daily_rows:,} rows")
    
    return files_created

# Generate sample data files
data_files = generate_production_data()
print(f"\nGenerated {len(data_files)} data files for ETL processing")

## Spark ETL Pipeline - Built for Production

Spark excels in production environments with built-in fault tolerance, monitoring, and distributed processing capabilities.

In [ ]:
# Initialize Spark session with production-like configuration
spark = SparkSession.builder \
    .appName("ProductionETL") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

# Define schema for data validation
transaction_schema = StructType([
    StructField("order_id", StringType(), False),
    StructField("customer_id", StringType(), False),
    StructField("product_id", StringType(), False),
    StructField("product_category", StringType(), False),
    StructField("order_total", DoubleType(), True),
    StructField("order_date", TimestampType(), False),
    StructField("processing_batch", StringType(), False)
])

def spark_etl_pipeline():
    """Production ETL pipeline with Spark"""
    start_time = time.time()
    
    try:
        # Read with schema enforcement and error handling
        df = spark.read \
            .option("header", "true") \
            .option("timestampFormat", "yyyy-MM-dd HH:mm:ss") \
            .schema(transaction_schema) \
            .csv("data/transactions/2025/01/*.csv")
        
        print(f"Read {df.count():,} total records")
        
        # Data quality checks
        invalid_customers = df.filter(col("customer_id").startswith("INVALID")).count()
        null_totals = df.filter(col("order_total").isNull()).count()
        
        print(f"Data quality issues found:")
        print(f"  - Invalid customers: {invalid_customers:,}")
        print(f"  - Null order totals: {null_totals:,}")
        
        # Clean data and add derived columns
        cleaned_df = df.filter(
            (~col("customer_id").startswith("INVALID")) & 
            (col("order_total").isNotNull())
        )
        
        # Complex transformations with built-in fault tolerance
        processed = cleaned_df \
            .withColumn("processing_date", current_date()) \
            .withColumn("is_weekend", dayofweek(col("order_date")).isin([1, 7])) \
            .withColumn("order_hour", hour(col("order_date"))) \
            .withColumn("order_month", month(col("order_date"))) \
            .groupBy("customer_id", "product_category", "processing_batch") \
            .agg(
                sum("order_total").alias("category_total"),
                countDistinct("order_id").alias("order_count"),
                avg("order_total").alias("avg_order_value"),
                collect_list("product_id").alias("products_purchased"),
                sum(when(col("is_weekend"), 1).otherwise(0)).alias("weekend_orders")
            ) \
            .cache()  # Cache for multiple downstream operations
        
        result_count = processed.count()
        print(f"Processed {result_count:,} customer-category combinations")
        
        # Write results with partitioning (simulated)
        processed.coalesce(10).write \
            .mode("overwrite") \
            .option("compression", "snappy") \
            .parquet("output/spark_customer_analytics")
        
        processing_time = time.time() - start_time
        print(f"Spark ETL completed in {processing_time:.2f} seconds")
        
        return processed.toPandas().head(10)  # Return sample for display
        
    except Exception as e:
        logger.error(f"Spark ETL failed: {str(e)}")
        raise

# Run Spark ETL
spark_results = spark_etl_pipeline()
print("\nSample Spark ETL Results:")
spark_results

## Polars ETL Pipeline - High Performance with Custom Infrastructure

Polars offers excellent performance but requires building custom fault tolerance and monitoring.

In [ ]:
def polars_etl_pipeline():
    """High-performance ETL with custom error handling"""
    start_time = time.time()
    
    # Define schema for validation
    schema = {
        "order_id": pl.Utf8,
        "customer_id": pl.Utf8,
        "product_id": pl.Utf8,
        "product_category": pl.Utf8,
        "order_total": pl.Float64,
        "order_date": pl.Datetime,
        "processing_batch": pl.Utf8
    }
    
    # Read multiple files with custom error handling
    dfs = []
    failed_files = []
    
    for file_path in glob.glob("data/transactions/2025/01/*.csv"):
        try:
            df = pl.read_csv(
                file_path,
                schema=schema,
                try_parse_dates=True,
                null_values=["NULL", "null", "", "NA"]
            )
            dfs.append(df)
            print(f"Successfully processed {file_path}: {len(df):,} rows")
        except Exception as e:
            failed_files.append(file_path)
            logger.error(f"Failed to process {file_path}: {e}")
    
    if failed_files:
        print(f"Warning: {len(failed_files)} files failed to process")
    
    # Concatenate all successful files
    combined = pl.concat(dfs)
    print(f"Combined dataset: {len(combined):,} rows")
    
    # Data quality checks and cleaning
    invalid_customers = combined.filter(pl.col("customer_id").str.starts_with("INVALID")).height
    null_totals = combined.filter(pl.col("order_total").is_null()).height
    
    print(f"Data quality issues:")
    print(f"  - Invalid customers: {invalid_customers:,}")
    print(f"  - Null order totals: {null_totals:,}")
    
    # Clean and transform data
    processed = (
        combined
        .filter(
            (~pl.col("customer_id").str.starts_with("INVALID")) &
            (pl.col("order_total").is_not_null())
        )
        .with_columns([
            pl.col("order_date").dt.weekday().is_in([6, 7]).alias("is_weekend"),
            pl.col("order_date").dt.hour().alias("order_hour"),
            pl.col("order_date").dt.month().alias("order_month")
        ])
        .group_by(["customer_id", "product_category", "processing_batch"])
        .agg([
            pl.col("order_total").sum().alias("category_total"),
            pl.col("order_id").n_unique().alias("order_count"),
            pl.col("order_total").mean().alias("avg_order_value"),
            pl.col("product_id").implode().alias("products_purchased"),
            pl.col("is_weekend").sum().alias("weekend_orders")
        ])
    )
    
    print(f"Processed {len(processed):,} customer-category combinations")
    
    # Write results (custom partitioning would be needed for production)
    processed.write_parquet("output/polars_customer_analytics.parquet")
    
    processing_time = time.time() - start_time
    print(f"Polars ETL completed in {processing_time:.2f} seconds")
    
    return processed.head(10).to_pandas()  # Convert to pandas for display

# Run Polars ETL
polars_results = polars_etl_pipeline()
print("\nSample Polars ETL Results:")
polars_results

## DuckDB ETL Pipeline - SQL-First Approach

DuckDB provides excellent analytical performance with familiar SQL syntax, though with limited distributed processing.

In [ ]:
def duckdb_etl_pipeline():
    """SQL-first ETL pipeline with DuckDB"""
    start_time = time.time()
    
    conn = duckdb.connect(":memory:")
    
    try:
        # Read all CSV files into a single table
        conn.execute("""
            CREATE TABLE raw_transactions AS 
            SELECT * FROM read_csv_auto('data/transactions/2025/01/*.csv')
        """)
        
        total_rows = conn.execute("SELECT COUNT(*) FROM raw_transactions").fetchone()[0]
        print(f"Loaded {total_rows:,} total records")
        
        # Data quality analysis
        quality_check = conn.execute("""
            SELECT 
                COUNT(*) FILTER (WHERE customer_id LIKE 'INVALID%') as invalid_customers,
                COUNT(*) FILTER (WHERE order_total IS NULL) as null_totals,
                COUNT(*) as total_records
            FROM raw_transactions
        """).fetchone()
        
        print(f"Data quality issues:")
        print(f"  - Invalid customers: {quality_check[0]:,}")
        print(f"  - Null order totals: {quality_check[1]:,}")
        
        # Complex ETL transformation with SQL
        conn.execute("""
            CREATE TABLE processed_analytics AS
            SELECT 
                customer_id,
                product_category,
                processing_batch,
                SUM(order_total) as category_total,
                COUNT(DISTINCT order_id) as order_count,
                AVG(order_total) as avg_order_value,
                LIST(DISTINCT product_id) as products_purchased,
                SUM(CASE WHEN EXTRACT(DOW FROM order_date) IN (0, 6) THEN 1 ELSE 0 END) as weekend_orders,
                CURRENT_DATE as processing_date
            FROM raw_transactions
            WHERE customer_id NOT LIKE 'INVALID%' 
              AND order_total IS NOT NULL
            GROUP BY customer_id, product_category, processing_batch
        """)
        
        result_count = conn.execute("SELECT COUNT(*) FROM processed_analytics").fetchone()[0]
        print(f"Processed {result_count:,} customer-category combinations")
        
        # Export results
        conn.execute("""
            COPY processed_analytics TO 'output/duckdb_customer_analytics.parquet' 
            (FORMAT PARQUET, COMPRESSION SNAPPY)
        """)
        
        processing_time = time.time() - start_time
        print(f"DuckDB ETL completed in {processing_time:.2f} seconds")
        
        # Return sample results
        return conn.execute("SELECT * FROM processed_analytics LIMIT 10").df()
        
    except Exception as e:
        logger.error(f"DuckDB ETL failed: {str(e)}")
        raise
    finally:
        conn.close()

# Run DuckDB ETL
duckdb_results = duckdb_etl_pipeline()
print("\nSample DuckDB ETL Results:")
duckdb_results

## Pandas ETL Pipeline - Traditional Approach

Pandas struggles with production ETL due to memory limitations and lack of built-in fault tolerance, but we'll demonstrate the approach.

In [ ]:
def pandas_etl_pipeline():
    """Traditional ETL with pandas - limited scalability"""
    start_time = time.time()
    
    try:
        # Read files with basic error handling
        dfs = []
        failed_files = []
        
        for file_path in glob.glob("data/transactions/2025/01/*.csv"):
            try:
                df = pd.read_csv(
                    file_path,
                    parse_dates=['order_date'],
                    dtype={
                        'order_id': 'string',
                        'customer_id': 'string',
                        'product_id': 'string',
                        'product_category': 'string',
                        'processing_batch': 'string'
                    }
                )
                dfs.append(df)
                print(f"Loaded {file_path}: {len(df):,} rows")
            except Exception as e:
                failed_files.append(file_path)
                logger.error(f"Failed to process {file_path}: {e}")
        
        if failed_files:
            print(f"Warning: {len(failed_files)} files failed")
        
        # Concatenate (memory intensive)
        combined = pd.concat(dfs, ignore_index=True)
        print(f"Combined dataset: {len(combined):,} rows")
        print(f"Memory usage: {combined.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
        
        # Data quality checks
        invalid_customers = combined[combined['customer_id'].str.startswith('INVALID', na=False)].shape[0]
        null_totals = combined['order_total'].isnull().sum()
        
        print(f"Data quality issues:")
        print(f"  - Invalid customers: {invalid_customers:,}")
        print(f"  - Null order totals: {null_totals:,}")
        
        # Clean and transform
        cleaned = combined[
            (~combined['customer_id'].str.startswith('INVALID', na=False)) &
            (combined['order_total'].notna())
        ].copy()
        
        # Add derived columns
        cleaned['is_weekend'] = cleaned['order_date'].dt.dayofweek.isin([5, 6])
        cleaned['order_hour'] = cleaned['order_date'].dt.hour
        cleaned['order_month'] = cleaned['order_date'].dt.month
        
        # Aggregation (memory intensive for large datasets)
        processed = (
            cleaned.groupby(['customer_id', 'product_category', 'processing_batch'])
            .agg({
                'order_total': ['sum', 'mean'],
                'order_id': 'nunique',
                'product_id': lambda x: list(x.unique()),
                'is_weekend': 'sum'
            })
            .round(2)
        )
        
        # Flatten column names
        processed.columns = ['category_total', 'avg_order_value', 'order_count', 'products_purchased', 'weekend_orders']
        processed = processed.reset_index()
        
        print(f"Processed {len(processed):,} customer-category combinations")
        
        # Save results
        processed.to_parquet("output/pandas_customer_analytics.parquet", compression='snappy')
        
        processing_time = time.time() - start_time
        print(f"Pandas ETL completed in {processing_time:.2f} seconds")
        
        return processed.head(10)
        
    except MemoryError:
        print("ERROR: Pandas ran out of memory - this is why it's not suitable for production ETL at scale")
        return pd.DataFrame()  # Return empty DataFrame
    except Exception as e:
        logger.error(f"Pandas ETL failed: {str(e)}")
        raise

# Run Pandas ETL (may fail due to memory constraints)
try:
    pandas_results = pandas_etl_pipeline()
    print("\nSample Pandas ETL Results:")
    print(pandas_results)
except Exception as e:
    print(f"Pandas ETL failed as expected: {e}")

## Performance Comparison and Analysis

Let's compare the tools across key production ETL criteria:

In [ ]:
# Performance and capability comparison
import matplotlib.pyplot as plt

# Create comparison matrix
comparison_data = {
    'Tool': ['Spark', 'Polars', 'DuckDB', 'Pandas'],
    'Fault Tolerance': [5, 2, 3, 1],  # 1-5 scale
    'Scalability': [5, 4, 3, 1],
    'Performance': [4, 5, 4, 2],
    'Monitoring': [5, 2, 2, 1],
    'Ecosystem': [5, 3, 3, 5],
    'Production Ready': [5, 3, 4, 2]
}

comparison_df = pd.DataFrame(comparison_data)
print("Production ETL Capability Comparison (1-5 scale):")
print(comparison_df.set_index('Tool'))

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Radar chart for capabilities
categories = ['Fault Tolerance', 'Scalability', 'Performance', 'Monitoring', 'Ecosystem', 'Production Ready']
tools = comparison_df['Tool'].tolist()

# Bar chart for overall production readiness
overall_scores = comparison_df.set_index('Tool').mean(axis=1)
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

bars = ax1.bar(overall_scores.index, overall_scores.values, color=colors)
ax1.set_title('Overall Production ETL Readiness Score')
ax1.set_ylabel('Average Score (1-5)')
ax1.set_ylim(0, 5)

# Add value labels on bars
for bar, score in zip(bars, overall_scores.values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
             f'{score:.1f}', ha='center', va='bottom')

# Capability breakdown
capability_matrix = comparison_df.set_index('Tool').T
im = ax2.imshow(capability_matrix.values, cmap='RdYlGn', aspect='auto', vmin=1, vmax=5)
ax2.set_xticks(range(len(tools)))
ax2.set_xticklabels(tools)
ax2.set_yticks(range(len(categories)))
ax2.set_yticklabels(categories)
ax2.set_title('Detailed Capability Heatmap')

# Add text annotations
for i in range(len(categories)):
    for j in range(len(tools)):
        ax2.text(j, i, capability_matrix.iloc[i, j], ha='center', va='center', 
                color='white' if capability_matrix.iloc[i, j] < 3 else 'black')

plt.colorbar(im, ax=ax2, label='Capability Score (1-5)')
plt.tight_layout()
plt.show()

print("\n=== SCENARIO 2 CONCLUSIONS ===")
print("\n🏆 WINNER: Spark for Enterprise ETL")
print("   - Built-in fault tolerance and error recovery")
print("   - Comprehensive monitoring and logging")
print("   - Mature ecosystem with extensive tooling")
print("   - Handles schema evolution and data validation")

print("\n🥈 RUNNER-UP: Polars for High-Performance Custom ETL")
print("   - Excellent performance for batch processing")
print("   - Lower resource requirements than Spark")
print("   - Requires building custom infrastructure")

print("\n📊 DuckDB: Good for Medium-Scale Analytics ETL")
print("   - SQL-first approach familiar to analysts")
print("   - Excellent analytical performance")
print("   - Limited distributed processing capabilities")

print("\n⚠️  Pandas: Not Suitable for Production ETL")
print("   - Memory limitations prevent scaling")
print("   - No built-in fault tolerance")
print("   - Single-threaded processing")

In [ ]:
# Clean up Spark session
spark.stop()
print("Spark session stopped successfully")